In [3]:
from data_processing.audio_processor import AudioProcessor
from data_processing.spectrogram_processor import SpectrogramProcessor

In [5]:
from pydub.generators import Sine

#Creating a dummy sound file with some silences in it
sine_wave = Sine(440).to_audio_segment(duration=3000) 
sine_wave.export("test_audio.mp3", format="mp3")
print("Generated test_audio.mp3")

Generated test_audio.mp3


In [9]:
import os
from pydub import AudioSegment

import pandas as pd
metadata_df = pd.DataFrame()

processor = AudioProcessor(input_folder=".", target_sample_rate=44100, target_duration=3, metadata_df=metadata_df)

# Test File Paths
original_file = "test_audio.mp3"
converted_file = "test_audio.wav"

# Test 1: Convert to WAV
print("Testing conv_to_wav...")
processor.conv_to_wav(original_file)

if os.path.exists(converted_file):
    print("Conversion successful")
else:
    print("Conversion failed")

# Test 2: Remove Silences
print("Testing remove_silences...")

# Create a test file with silence + sine wave 
silence_segment = AudioSegment.silent(duration=2000)  # 2 seconds silence
sine_wave = Sine(440).to_audio_segment(duration=2000)  # 2 seconds sine wave

silent_audio = silence_segment + sine_wave  # Combine them
silent_audio.export("test_silent_audio.wav", format="wav")

# Test remove_silences()
processor.remove_silences("test_silent_audio.wav")

processed_audio = AudioSegment.from_file("test_silent_audio.wav")

# Check if silence was removed
if len(processed_audio) < len(silent_audio):
    print("Silence removal successful.")
else:
    print("Silence removal failed.")

print("Test completed.")

Testing conv_to_wav...
Converted test_audio.mp3 to test_audio.wav
Conversion successful
Testing remove_silences...
Removed silence from test_silent_audio.wav
Silence removal successful.
Test completed.
